In [ ]:
import polars as pl
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from utils import *

w_insect = 400
insect = np.loadtxt("insect15.txt")

w = 640
i, j = 643, 8724
motif_idxs = np.array([i, j], dtype=np.int64)

np.random.seed(1234)
steamgen = pl.read_csv("steamgen.csv")
steam = steamgen.select("steam flow").to_numpy()[:, 0]
sample_idxs = np.random.choice(
    np.arange(steam.shape[0]-w, dtype=np.int64), 20)
p_idxs = np.concatenate([motif_idxs, sample_idxs])

windows = sliding_window_view(steam, w)
swindows = windows[p_idxs, :]
embedder = PCA(n_components=2)
prj = embedder.fit_transform(swindows)
np.random.seed(1234)
prj = prj + np.random.normal(scale=8, size=prj.shape)
prj = MinMaxScaler(feature_range=(-0.5, 0.5)).fit_transform(prj)

## Outline

1. Problem definition
2. State of the art
3. Locality Sensitive Hashing
4. Our algorithm
5. Experimental results

## Motivation

<!-- TODO: Write motivation -->

## Subsequences {.smaller}

<!-- TODO: Introduce the Euclidean distance between subsequences -->

::::{.r-stack}
:::{}

In [ ]:
plot_ts(insect)

:::

:::{.fragment fragment-index=1}

In [ ]:
plot_ts(insect, w=w_insect, highlight=[200])

:::

:::{.fragment fragment-index=2}

In [ ]:
plot_ts(insect, w=w_insect, highlight=[200, 6974])

:::

:::{.fragment fragment-index=3}

In [ ]:
plot_ts(insect, w=w_insect, highlight=[200, 6974, 8490])

:::
::::

:::{.fragment fragment-index=1}
Fix a subsequence length $w$
:::

::::{.columns}
:::{.column .fragment fragment-index=2}

In [ ]:
plot_eucl(insect, w=w_insect, i=6974, j=200, ci="steelblue", cj="forestgreen")

:::

:::{.column .fragment fragment-index=3}

In [ ]:
plot_eucl(insect, w=w_insect, i=6974, j=8490, ci="steelblue", cj="orange")

:::
::::

. . .

- The Euclidean distance measures the _similarity_ between subsequences
- Subsequences can be seen as points in a $w$ dimensional space

## Top-$k$ motifs {.center}

<!-- TODO: Introduce the problem definition, with figures -->

## {.center}

![](imgs/steamgen-with-window.png)

## {.center}

![](imgs/steamgen-with-motifs.png)


## Top-$k$ motifs {.center}

Consider all pairs of (non-overlapping) subsequences of length $w$ of a given time series.
The first $k$ by increasing distance are the top-$k$ motifs.

## State of the art: Matrix Profile

<center>
![](imgs/steamgen-with-matrix-profile.png){width="80%"}
</center>

::: aside
Zimmerman et al.
_Scaling Time Series Motif Discovery with GPUs to Break a Quintillion Pairwise Comparisons a Day and Beyond._
SoCC19
:::

## State of the art: Matrix Profile {.smaller}

- For each subsequence, find the nearest neighbor
- Find the pair at minimum distance

. . .

- Leverage the fact that consecutive subsequences share a lot of structure
- Parallelize with many GPUs

. . .

- Still, it's a $\Omega(n^2)$ algorithm

- Finds motifs out of 1 billion long time series in one day, using 5 GPUs 

::: aside
Zimmerman et al.
_Scaling Time Series Motif Discovery with GPUs to Break a Quintillion Pairwise Comparisons a Day and Beyond._
SoCC19
:::

##

### Goal

- Find the top motifs without computing all-pairs-distances
- We need an index

:::{.fragment}

### Challenges

- Subsequences can be seen as vectors
- These vectors are high dimensional
- Curse of dimensionality: indices such as R-trees degrade to linear scans
:::

## {.center}

To address these challenges, we will use an approach based on
Locality Sensitive Hashing (LSH)

## {.smaller}

:::{.absolute top=0}

In [ ]:
plot_ts(steam)

:::


:::{.fragment .absolute top=0 fragment-index=1}

In [ ]:
plot_ts(steam, w=w, highlight=[i, j])
plt.savefig("imgs/steam-highlight.png", dpi=300)

:::

:::{.fragment .absolute top="30%" fragment-index=1}

In [ ]:
plot_projection_circle(prj, highlight=[0, 1])

:::

:::{.fragment .absolute top="40%" right=0 fragment-index=2}
We consider the
z-normalized Euclidean distance
$$
d(x, y) = \sqrt{\sum_{i\in[1,w]} \left( 
  \frac{x_i - \bar{x}}{\sigma_x}
  -
  \frac{y_i - \bar{y}}{\sigma_y}
\right)^2
}
$$
In this example we have a window length $w = 640$, <br />
hence we have
vectors in $\mathbb{R}^{640}$
:::

## {.smaller}

:::{.absolute top="20%"}

In [ ]:
plot_projection_circle(prj)

:::

:::{.fragment .absolute top="20%" fragment-index=1}

In [ ]:
plot_hashes(prj, k=1, r=0.25, seed=1234)
plt.savefig("imgs/steam-hashes-k1-l1.png", dpi=300)

:::

:::{.absolute top=10 left="54%"}
Locality Sensitive Hash function:

- choose $r \in \mathbb{R}^+$
- sample $\vec{a} \sim \mathcal{N}(0,1)^w$, $b \sim \mathcal{U}(0,r)$

$$
h(\vec{x}) = \left\lfloor\frac{\vec{a} \cdot \vec{x} + b}{r}\right\rfloor
$$
:::

:::{.fragment .absolute bottom="20%" left="54%" fragment-index=1}
The key point is that we only compute the distance 
of
subsequences falling <br/>into the same bucket.
:::

:::{.fragment .absolute bottom=0 left="54%" fragment-index=2}

In [ ]:
plot_cp(w, 1)
plt.savefig("imgs/steam-cp-k1-l1.png", dpi=300)

:::

## {.smaller}

:::{.absolute top="20%" fragment-index=1}

In [ ]:
plot_hashes(prj, k=2, r=0.25, seed=1234)
plt.savefig("imgs/steam-hashes-k2-l1.png", dpi=300)

:::

:::{.absolute top=10 left="54%"}
To lower the collision probability we concatenate $k$ hash functions
$$
\hat{h}(\vec{x}) = \langle h_1(\vec{x}), \dots, h_k(\vec{x}) \rangle
$$
this makes for a better _precision_ of the index.
:::


:::{.absolute bottom=0 left="54%" fragment-index=2}

In [ ]:
#plot_cp(w, [0.5,1,2])
plot_cp(w, 1, [1,2,3,4])
plt.savefig("imgs/steam-cp-kmany-l1.png", dpi=300)

:::

## {.smaller}

:::{.absolute top="0%" fragment-index=1}

In [ ]:
#| layout-ncol: 2
for i, seed in enumerate([1234, 243,256,3562,235,1]):
    plot_hashes(prj, k=2, r=0.25, seed=seed, size=2, title=f"Repetition {i+1}")
    plt.savefig(f"imgs/steam-hashes-k2-{seed}.png", dpi=300)

:::

:::{.absolute top=10 left="54%"}
And to increase the _recall_ of the index we repeat $L$ times.
:::


:::{.absolute bottom=0 left="54%" fragment-index=2}

In [ ]:
plot_success_p(w, 1, k=2, ls=[1, 10, 100], title="For k=2")
plt.savefig("imgs/steam-sp.png", dpi=300)

:::


## A simple algorithm {.smaller}

1. Build a priority queue to hold candidate motifs

2. Fix $K$

3. Repeat:
  1. Sample $K$ hash functions, hash all the points, and compute the distances of pairs in the same bucket
  2. Let $d_k$ be the distance of the $k$-th motif
  3. If the probability of having seen all pairs <br /> at distance $\ge d_k$ is at least $1-\delta$,<br/> then **return**.

:::{.absolute bottom=0 left="54%" .fragment}

In [ ]:
#plot_success_p(w, 1, k=2, ls=[1, 10, 100], title="For fixed k")
plot_success_by_reps(w, 1, k=2, dist=1, max_reps=50)

:::

## Dealing with parameters

- How to fix $K$?
- The wrong choice could force us to run too many repetitions

. . .

- We adopt an approach that auto-tunes parameters based on the data

## {.center}

- $L_{max}$ maximum number of repetitions,
- $K_{max}$ maximum number of concatenations (e.g. 4).

::::{.columns}
:::{.column .fragment}
![Repetition 1](imgs/example-rep1.png){width="100%"}
:::

:::{.column .fragment}
![Repetition 2](imgs/example-rep3.png){width="100%"}
:::
::::

## {.smaller .center}

In each iteration we compute the distance of all subsequences in the same
bucket.

::::{.r-stack}

<!---- k = 4 ------------------------>
:::{.bg-white}

In [ ]:
k=4
dist = 1
prevs = []
plot_execution(w, r=1, k=k, max_reps=100, rep=1, p_threshold=0.9, dist=dist + 1, prevs=prevs)
prevs.append((k, 1))

In the first iteration, with $k=4$, we discover a pair at distance 2<br/>
<span class="text-white">.</span>
:::
:::{.fragment .bg-white}

In [ ]:
plot_execution(w, r=1, k=k, max_reps=100, rep=10, p_threshold=0.9, dist=dist, prevs=prevs)
prevs.append((k, 10))

After 10 repetitions, we find a pair at distance 1<br/>
<span class="text-white">.</span>
:::
:::{.fragment .bg-white}

In [ ]:
plot_execution(w, r=1, k=k, max_reps=100, rep=100, p_threshold=0.9, dist=dist, prevs=prevs)
prevs.append((k, 100))

After 100 repetitions we did not find a better pair,<br/> and the success probability 
is about 0.85
:::
<!---- k = 3 ------------------------>
:::{.fragment .bg-white}

In [ ]:
k=3
plot_execution(w, r=1, k=k, max_reps=100, rep=1, p_threshold=0.9, dist=dist, prevs=prevs)
prevs.append((k, 1))

We then consider shorter prefixes of the hashes, <br/>going through the 100
repetitions again.
:::
:::{.fragment .bg-white}

In [ ]:
plot_execution(w, r=1, k=k, max_reps=100, rep=15, p_threshold=0.9, dist=dist, prevs=prevs)
prevs.append((k, 15))

After 15 repetitions, we observe that the<br/> success probability is above our
target, and thus return.
:::

::::


## Complexity {.smaller}

:::{#thm-index}
The LSH index construction takes time
$$
O(K_{max} \cdot \sqrt{L_{max}} n\log n)
$$
:::


:::{#thm-motif}
Let $d(m_k)$ be the distance of the $k$-th motif,
and $i'\le K$, $j' \le L$ be the parameters used
by the optimal LSH algorithm.
Then, the algorithm considers
$$
O\left(
j'\sum_{m\in T^w\times T^w} p(d(m))^{i'}
+
(L-j')\sum_{m\in T^w\times T^w} p(d(m))^{i'+1}
\right)
$$
pairs in expectation.
:::

::: notes
The important thing is that Theorem #thm-motif is dependent on the distribution of distances: if we look at a lot of pairs that is because the distribution of data is difficult.
:::


## Optimizations

- Use a trie data-structure to re-use computations across iterations at different $k$ values
- Compute dot producs for hash values in the frequency domain (also done in some implementations of the Matrix Profile)
- Compute fewer hash values using _tensoring_


## Experimental results {.smaller}

:::::{.columns}

::::{.column}
Find the top-10 motifs.

:::{style="font-size: 1.3rem;"}
| dataset | $n$  (millions) | RC |
|:--------|----:|----------------:|
| astro   | 1  |  8.63 |
| GAP     | 2  | 9.17 |
| freezer | 7  | 7.95 |
| ECG     | 7  | 109.06 |
| HumanY  | 26 | 581.03 |
| Whales  | 308 | 21.66 |
| Seismic | 1000 | 274.44 |
:::

Relative Contrast measures difficulty: higher is easier.
$$
RC = \frac{d_{avg}}{d_{m_k}}
$$
::::

::::{.column .fragment}
![](imgs/10-motifs.png){width="100%"}
::::

:::::

## Scalability {.smaller}

![Synthetic data, planted motifs with different relative contrasts. `SCAMP-gpu` only has one line since it is data-independent.](imgs/scalability_n.png){width="100%"}

## Practical takeaways

:::{.incremental}

- For shorter time series, or if the relative contrast is very small, use the
  Matrix Profile.

- For time series of a few million values and above, with a not-to-small relative
  contrast, try `Attimo`
:::

## References {.smaller}

- Matteo Ceccarello, Johann Gamper:
_Fast and Scalable Mining of Time Series Motifs with Probabilistic Guarantees._
Proc. VLDB Endow. 15(13): 3841-3853 (2022)

::::{.columns}
:::{.column}
<center>
https://github.com/Cecca/attimo

![](imgs/qr-attimo-repo.png)
</center>
:::
:::{.column}
<br/>
<br/>

In [ ]:
#| echo: true
import pyattimo
ts = pyattimo.load_dataset("ecg", prefix=1000000)
motifs = pyattimo.MotifsIterator(
    ts, 
    w=1000
)
m = next(motifs)
print(m)

:::
::::

<center>
  `matteo.ceccarello@unipd.it`
</center>

## Appendix

## Influence of the maximum number of repetitions

![Running for top-10 motifs, for different number of repetitions.](imgs/repetitions.png)

## Difficult datasets {.smaller}

Data from LIGO:


In [ ]:
case1 = np.loadtxt("imgs/case1.txt")
plot_ts(case1[:2000], w=1000, highlight=[100])

::::{.columns}
:::{.column}
- Length 100k, window 1k
- Top motif distance $\approx 40$
- Average distance $\approx 44$
- Relative contrast $\approx 1.1$
:::
:::{.column}
- `Attimo`: 6 hours
- `SCAMP`: $\approx 1$ second
:::
::::

:::{.aside}
Thanks to Eamonn Keogh and Nathan Romanelli for providing and discussing this data.
:::


## `freezer` and the 7-th motif {.smaller}

7M points, window 5000


In [ ]:
w = 5000
freezer = np.loadtxt("../../data/freezer.txt.gz")
motifs_mp = [
  (3705031, 1834102, 4.195242485),
  (3698075, 4733298, 5.765751866),
  (2352371, 4186995, 7.077046765),
  (4002563, 3993450, 7.318316307),
  (4618976, 4812738, 9.207241828),
  (1825969, 1993859, 9.366285725),
  (1697587, 1408089, 10.56533893),
  (5169982, 6429402, 11.46242184),
  (6641806, 5230708, 12.46052708),
  (6339277, 191377, 12.50717434),
  (36578, 3988437, 12.73866733)
]
i, j = (3815625, 5170040)
d = zeucl(freezer[i:i+w], freezer[j:j+w])

plt.figure(figsize=(14,3))
plt.axhline(0, 0, 13, c="gray")

last = 0
for ii, jj, dd in motifs_mp:
    off = 0.04 if dd - last > 0.6 else 0.08
    if dd > 12.7:
        off = 0.12      
    last = dd
    plt.plot([dd,dd], [0, off], c="gray", linewidth=0.2)
    plt.scatter(dd, 0, c="black")  
    plt.text(dd, off, f"{dd:.3f}", fontsize=14, va="top")

off = -0.06
plt.plot([d, d], [0, off], c="firebrick")
plt.scatter(d, 0, c="firebrick")  
plt.text(d, off, f"{d:.3f}", color="firebrick", fontsize=14, va="bottom")

height = 0.16
plt.gca().set_ylim((-height, height))

plt.axis("off")
plt.show()

- In black are the distances of the top-10 motifs extracted from the matrix
  profile.
- In red the distance of a pair of subsequences neither of which is the nearest
  neighbor of the other, and not overlapping with higher-ranked motifs.
- The matrix profile holds the distances and indices of the 1-nearest neighbor
  of each subsequence, but top-k motifs would require the k-nearest neighbors
  to be maintained in the matrix profile.

## Top-k and k-NN {.smaller}

::::{.columns}
:::{.column}

In [ ]:
#| out-width: "100%"
points = np.array([
  (0,0),
  (2,0),
  (0.3, 0.3),
  (2-0.3, 0.3)
])
shapes = [
  "o", "o",
  "D", "D"
]
colors = [
  "darkorange", "darkorange",
  "blue", "blue"
]
plt.figure(figsize=(5,5))
order = [0,2,3,1]
plt.plot(points[order, 0], points[order,1], linestyle="-", c="black", zorder=0)
plt.plot(points[[0,1], 0], points[[0,1],1], linestyle="--", c="firebrick", zorder=0)

for i in range(points.shape[0]):
    plt.scatter(points[i,0], points[i,1], c=colors[i], marker=shapes[i], zorder=1)

plt.gca().set_aspect('equal')
plt.axis("off")
plt.gca().set_ylim((-0.1, 0.4))
plt.tight_layout()
plt.show()

:::
:::{.column}
- Solid lines are nearest neighbor distances
- The dashed line is the distance of the top-2 pair in the definition we are using
- The Matrix Profile contains information about 1-nearest neighbors (solid black lines)
:::
::::
